This script will generate fasta files from vcf files generated from slim in order to use those fasta fiels to generate reads, that would be the ouput of poolseq 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as statsß
import math 
import os

import allel
import subprocess

In [28]:
def from_vcf_to_fasta(vcf_file_name, path_vcf, path_fasta):
    vcf = allel.read_vcf(path_vcf + vcf_file_name)
    samples = vcf['samples']

    ## first, make sure that if the file exists, delete it because otherwise it will add lines to an existing file 
    if os.path.exists(path_fasta + vcf_file_name[:-4] + '.fasta'):
        os.remove(path_fasta + vcf_file_name[:-4] + '.fasta')

## will just do it for 20 
    for individual in samples: 
        #print(individual)
        vcf = allel.read_vcf('vcf_slim/subp0.vcf', samples=[individual])
        gt = vcf['calldata/GT']
        ref = vcf['variants/REF']
        alt = vcf['variants/ALT']

        ## i will only keep the ones in the first column for the alt 
        alt = alt[:, 0] 
        first_chr = gt[:, 0, 0]
        second_chr = gt[:, 0, 1]

        # Retrieve the reference sequence for the genomic region covered by the VCF file

        ## here i will have to pretend that each organisms is a chromosome 
        #chrom = vcf['variants/CHROM'][0]
        start = vcf['variants/POS'][0]
        end = vcf['variants/POS'][-1]
        # Convert the genotype information to a nucleotide sequence
        seq_firstchr = ''
        for i in range(0, len(first_chr)):
            if first_chr[i] == 0:
                seq_firstchr += ref[i]
            else:
                seq_firstchr += alt[i]
        seq_firstchr
        with open(path_fasta + vcf_file_name[:-4] + '.fasta', 'a') as f:
            f.write(f'>{individual}:{start}-{end}\n')
            f.write(seq_firstchr + '\n')

In [29]:
from_vcf_to_fasta('subp0.vcf', 'vcf_slim/','vcf_slim/fasta_slim/')

In [5]:
# take all the vcf files in the folder 
subp = os.listdir( 'vcf_slim/')
subp = [i for i in subp if '.vcf' in i ]

In [6]:
## generate the fasta files for all the subpopulations 
for files in subp:  
    from_vcf_to_fasta(files, 'vcf_slim/','vcf_slim/fasta_slim/')

NameError: name 'from_vcf_to_fasta' is not defined

In [30]:
## checking that is has the right amount of lines 
!wc -l vcf_slim/fasta_slim/subp0.fasta 

    1000 vcf_slim/fasta_slim/subp0.fasta


In [16]:
fasta = os.listdir( 'vcf_slim/fasta_slim/')
fasta = [i for i in fasta if '.fasta' in i ]

In [18]:
str(fasta).replace(',','').replace('\'','')

'[subp1.fasta subp3.fasta subp15.fasta subp11.fasta subp7.fasta subp5.fasta subp13.fasta subp2.fasta subp14.fasta subp0.fasta subp4.fasta subp12.fasta subp10.fasta subp6.fasta subp8.fasta subp9.fasta]'

In [20]:
from Bio import SeqIO

fasta_file = "vcf_slim/fasta_slim/subp0.fasta"

In [21]:
with open(fasta_file, "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        allele = record.id
        seq_length = len(record.seq)
        print("Allele: {}\tLength: {}".format(allele, seq_length))

Allele: i0:6063-30411563	Length: 4698
Allele: i1:6063-30411563	Length: 4698
Allele: i2:6063-30411563	Length: 4698
Allele: i3:6063-30411563	Length: 4698
Allele: i4:6063-30411563	Length: 4698
Allele: i5:6063-30411563	Length: 4698
Allele: i6:6063-30411563	Length: 4698
Allele: i7:6063-30411563	Length: 4698
Allele: i8:6063-30411563	Length: 4698
Allele: i9:6063-30411563	Length: 4698
Allele: i10:6063-30411563	Length: 4698
Allele: i11:6063-30411563	Length: 4698
Allele: i12:6063-30411563	Length: 4698
Allele: i13:6063-30411563	Length: 4698
Allele: i14:6063-30411563	Length: 4698
Allele: i15:6063-30411563	Length: 4698
Allele: i16:6063-30411563	Length: 4698
Allele: i17:6063-30411563	Length: 4698
Allele: i18:6063-30411563	Length: 4698
Allele: i19:6063-30411563	Length: 4698
Allele: i20:6063-30411563	Length: 4698
Allele: i21:6063-30411563	Length: 4698
Allele: i22:6063-30411563	Length: 4698
Allele: i23:6063-30411563	Length: 4698
Allele: i24:6063-30411563	Length: 4698
Allele: i25:6063-30411563	Length: 4

In [ ]:
## ok based on this information, before actually using the reads i should revisit this code, since it is producing reads from vc file not accounting vor not variant sites 